# Handling tool call errors

## LangChain Tool Error Handling

Exceptions raised inside tool functions propagate up and cause the entire agent graph invocation to error out. The graph does not automatically catch and relay exceptions to the agent for self-correction.

### Expected Errors → Return Error Messages
For errors the agent can reasonably recover from (validation failures, missing data, user input issues), return a serializable error message instead of raising an exception. This keeps the agent loop running, allowing the LLM to read the error and iterate on its approach.

### Unexpected Errors → Let Them Raise
For errors indicating genuine failures (network issues, bugs, resource exhaustion), let exceptions propagate to break the graph.

### Decision Framework
| Error Type | Example |Action |
| ---------- | ------- | ----- |
| Validation/input errors | Bad query syntax, missing fields | Return message |
| Empty/no results | Query found nothing | Return message |
| Transient failures | Rate limits (if retryable) | Return message |
| Infrastructure failures | DB down, network errors | Raise exception |
| Programming errors | Bugs, type errors | Raise exception |
| Resource exhaustion | OOM, disk full | Raise exception |

In [ ]:
import inspect

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage
from langchain.tools import tool
from loguru import logger
from pydantic import BaseModel, Field

from chain_reaction.config import APIKeys, ModelBehavior, ModelName

# Define a flaky tool

In [ ]:
@tool
def add_two_integers(a: int, b: int) -> int:
    """Add two integers and return the result.

    Args:
        a (int): The first integer to add.
        b (int): The second integer to add.

    Returns:
        int: The sum of the two integers.

    Raises:
        ValueError: If the first integer is less than the second integer.
    """
    logger.debug(f"Adding two integers: a={a}, b={b}")
    if a < b:
        logger.error("The first integer must be greater than or equal to the second integer.")
        raise ValueError(
            "The first integer must be greater than or equal to the second integer."
            f" Called with a={a}, b={b}."
            f" Instead, try calling add_two_integers(a={b}, b={a})"
        )
    return a + b

In [ ]:
# This should work
result = add_two_integers.invoke({"a": 5, "b": 3})
assert result == 8  # noqa: S101

In [ ]:
# This should raise a ValueError
try:
    add_two_integers.invoke({"a": 2, "b": 4})
except ValueError as e:
    print(e)

# Define an agent with tool

In [ ]:
# Initialize a chat model with your API key
chat_model = init_chat_model(
    model=ModelName.CLAUDE_HAIKU,
    timeout=None,
    max_retries=2,
    api_key=APIKeys().anthropic,
    **ModelBehavior.factual().model_dump(),
)


# Create a response model
class CalculationResult(BaseModel):
    """Response model for calculation results."""

    result: int = Field(description="The result of the calculation.")


# Initialize an agent using the chat model & tools
agent = create_agent(
    model=chat_model,
    tools=[add_two_integers],
    system_prompt="""
    You're a helpful assistant that can perform mathematical calculations.
    Use the provided calculation tools to answer user questions accurately.
    """,
    response_format=CalculationResult,
)

# Invoke agent

In [ ]:
# This should work
response = agent.invoke(input={"messages": [HumanMessage(content="What's 5 plus 3?")]})
calc_result: CalculationResult = response["structured_response"]
calc_result

In [ ]:
# This should fail
response = agent.invoke(input={"messages": [HumanMessage(content="What's 2 plus 3?")]})
calc_result: CalculationResult = response["structured_response"]
calc_result

# Redefine tool to return error message

In [ ]:
# Define a structured error model to provide context on tool call failures
class ToolCallError(BaseModel):
    """Custom error model for providing context on expected tool call errors."""

    tool_name: str = Field(description="Name of the tool that failed.")
    error_message: str = Field(description="Error message describing the tool call failure.")


@tool
def add_two_integers_with_error_handling(a: int, b: int) -> int | ToolCallError:
    """Add two integers and return the result.

    Args:
        a (int): The first integer to add.
        b (int): The second integer to add.

    Returns:
        int | ToolCallError: Sum of the two integers or an error if the first integer is less than the second integer.
    """
    logger.debug(f"Adding two integers: a={a}, b={b}")
    if a < b:
        logger.error("The first integer must be greater than or equal to the second integer.")
        return ToolCallError(
            tool_name=inspect.currentframe().f_code.co_name,
            message=(
                "The first integer must be greater than or equal to the second integer."
                f" Called with a={a}, b={b}."
                f" Instead, try calling with a={b}, b={a}"
            ),
        )
    return a + b

In [ ]:
# This should work
result = add_two_integers_with_error_handling.invoke({"a": 5, "b": 3})
assert result == 8  # noqa: S101

In [ ]:
# This should return a ToolCallError
result = add_two_integers_with_error_handling.invoke({"a": 2, "b": 3})
assert isinstance(result, ToolCallError)  # noqa: S101
print(result)

In [ ]:
# Initialize an agent using the chat model & tools
agent = create_agent(
    model=chat_model,
    tools=[add_two_integers_with_error_handling],
    system_prompt="""
    You're a helpful assistant that can perform mathematical calculations.
    Use the provided calculation tools to answer user questions accurately.
    """,
    response_format=CalculationResult,
)

In [ ]:
# This should work
response = agent.invoke(input={"messages": [HumanMessage(content="What's 5 plus 3?")]})
calc_result: CalculationResult = response["structured_response"]
calc_result

In [ ]:
# This should also work
response = agent.invoke(input={"messages": [HumanMessage(content="What's 2 plus 3?")]})
calc_result: CalculationResult = response["structured_response"]
calc_result